# Calculate and visualise the geospatial centre of carbon dioxide emissions over time

This notebook shows the process of visualising the geospatial centre of carbin dioxide emissions over time. The procedure is loosely:
- Load and combine two data sets to get the country centroids, year, and CO$_2$ emissions.
- Calculate the weighted mean coordinate of CO$_2$ emissions.
- Visualise the data in an animation.

In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, atan2, asin, pi
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.feature import COASTLINE, OCEAN, LAND, BORDERS
from IPython.display import display, clear_output, HTML
import pycountry_convert as pcc

In [2]:
%matplotlib notebook

## Import and combine the data sets
Load country centroids. The data is from https://atcoordinates.info/resources/ 'Country Centroids'. The column `ISO3136` is erroneously named; it should be `ISO3166`, as it refers to the ISO 3166-1 Alpha-2 country code.

In [3]:
df_coords = pd.read_csv(
    './data/country_centroids_all.csv',
    sep='\t'
)
df_coords

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
0,33.000000,66.00,330000,660000,42STB1970055286,NI42-09,PCLI,NaN,AF,Afghanistan,Islamic Republic of Afghanistan,2009-04-10,AF
1,41.000000,20.00,410000,200000,34TDL1589839239,NK34-08,PCLI,NaN,AL,Albania,Republic of Albania,2007-02-28,AL
2,28.000000,3.00,280000,30000,31REL0000097202,NH31-15,PCLI,NaN,AG,Algeria,People's Democratic Republic of Algeria,2011-03-03,DZ
3,-14.333333,-170.00,-142000,-1700000,1802701,NaN,PCLD,US,AS,American Samoa,Territory of American Samoa,1998-10-06,AS
4,42.500000,1.50,423000,13000,31TCH7675006383,NK31-04,PCLI,NaN,AN,Andorra,Principality of Andorra,2007-02-28,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,31.666667,35.25,314000,351500,36RYA1331505689,NH36-04,TERR,NaN,WE,West Bank,West Bank,1993-12-16,PS
253,25.000000,-13.50,250000,-133000,28RFN5137665785,NG28-12,PCL,NaN,WI,Western Sahara,Western Sahara,2007/02/28,EH
254,15.500000,47.50,153000,473000,38PQC6820715194,ND38-04,PCLI,NaN,YM,Yemen,Republic of Yemen,2010-12-09,YE
255,-15.000000,30.00,-150000,300000,36LSJ7734939486,SD36-09,PCLI,NaN,ZA,Zambia,Republic of Zambia,2007-02-28,ZM


Now load emissions data, which is from https://github.com/owid/co2-data.

In [4]:
df_emissions = pd.read_csv('./data/owid-co2-data.csv')
df_emissions

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.00,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.70,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.00,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.00,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25199,ZWE,Zimbabwe,2016,10.738,12.153,-12.17,-1.488,1.415,0.765,0.866,...,4.703,11.92,0.85,6.55,0.467,14030338.0,2.096179e+10,47.5,3385.574,1.889
25200,ZWE,Zimbabwe,2017,9.582,11.248,-10.77,-1.156,1.666,0.673,0.790,...,NaN,NaN,NaN,NaN,NaN,14236599.0,2.194784e+10,NaN,NaN,NaN
25201,ZWE,Zimbabwe,2018,11.854,13.163,23.72,2.273,1.308,0.821,0.912,...,NaN,NaN,NaN,NaN,NaN,14438812.0,2.271535e+10,NaN,NaN,NaN
25202,ZWE,Zimbabwe,2019,10.949,12.422,-7.64,-0.905,1.473,0.748,0.848,...,NaN,NaN,NaN,NaN,NaN,14645473.0,NaN,NaN,NaN,NaN


Construct a new data frame with ISO 3166-1 Alpha-2 country code, lat-long coordinates, year, and CO2 emissions.

To get the latitude-longitude coordinates from `df_coords`, the Alpha-3 country codes in the `df_emissions` dataframe are converted to Alpha-2 codes and the values are looked up in the `df_coords` dataframe.

In [5]:
tot_rows = len(df_emissions.index)
pad = len(str(tot_rows))

data = []

for i, row in df_emissions.iterrows():
    store = False
    if i % 100 == 0 or i == tot_rows-1:
        clear_output(wait=True)
        display(f'Row {i:0{pad}}/{tot_rows-1}')
    
    # Extract the ISO 3166-1 Alpha-3 country code
    alpha3 = row['iso_code']
    
    # Get latitude-longitude coordinates
    if str(alpha3) not in ['OWID_KOS', 'OWID_WRL', 'nan']:
        alpha2 = pcc.country_alpha3_to_country_alpha2(alpha3)
        try:
            lat = df_coords[df_coords['ISO3136']==alpha2]['LAT'].values[0]
            long = df_coords[df_coords['ISO3136']==alpha2]['LONG'].values[0]
            store = True
        except IndexError:
            store = False
    # Custom rule for Kosovo
    elif str(alpha3) == 'OWID_KOS':
        alpha2 = 'KOS'
        lat = 42.583333
        long = 21.0
        store = True
    
    # Get the year
    year = row['year']
    
    # Get the CO2 emissions
    CO2 = row['co2']
    
    if store:
        data.append([alpha2, lat, long, year, CO2])

'Row 25203/25203'

In [6]:
df_data = pd.DataFrame(data, columns=['ISO', 'LAT', 'LONG', 'YEAR', 'CO2'])
df_data

,ISO,LAT,LONG,YEAR,CO2
0,AF,33.0,66.0,1949,0.015
1,AF,33.0,66.0,1950,0.084
2,AF,33.0,66.0,1951,0.092
3,AF,33.0,66.0,1952,0.092
4,AF,33.0,66.0,1953,0.106
...,...,...,...,...,...
21607,ZW,-19.0,29.0,2016,10.738
21608,ZW,-19.0,29.0,2017,9.582
21609,ZW,-19.0,29.0,2018,11.854
21610,ZW,-19.0,29.0,2019,10.949


## Create the time series
First need to define some functions for converting between coordinate frames and calculating the weighted average.

In [7]:
def convert_lat_long_to_cartesian(lat, long):
    """Convert latitude-longitude coordinate pair to Cartesian coordinates.
    
    Arguments
    ---------
    lat : float
        Latitude coordinate in degrees.
    long : float
        Longitude coordinate in degrees.
    
    Returns
    -------
    Length 3 tuple of (x, y, z) Cartesian coordinates.
    """
    # Convert to radians
    lat = lat * pi / 180
    long = long * pi / 180
    
    # Calculate Cartesian coordinates
    x = cos(lat) * cos(long)
    y = cos(lat) * sin(long)
    z = sin(lat)
    return (x, y, z)

In [8]:
def convert_cartesian_to_lat_long(x, y, z):
    """Convert Cartesian coordinates to latitude-longitude.
    
    Arguments
    ---------
    x, y, z : floats
        Cartesian coordinates.
    
    Returns
    Length 2 tuple of (lat, long) coordinates in degrees.
    """
    lat = asin(z)
    long = atan2(y, x)
    return (lat*180/pi, long*180/pi)

In [9]:
def weighted_average_coordinates(x_vals, y_vals, z_vals, w_vals):
    """Compute weighted average of coordinates.
    
    Arguments
    ---------
    x_vals, y_vals, z_vals : lists
        Lists of x-, y- and z-coordinate values.
    w_vals : list
        List of weights.
    
    Returns
    -------
    Length 3 tuple of Cartesian coordinates (x, y, z).
    """
    x_vals = np.array(x_vals)
    y_vals = np.array(y_vals)
    z_vals = np.array(z_vals)
    w_vals = np.array(w_vals)
    
    # Filter out nans
    x_nan = np.isnan(x_vals)
    y_nan = np.isnan(y_vals)
    z_nan = np.isnan(z_vals)
    w_nan = np.isnan(w_vals)
    mask = ~np.logical_or.reduce((x_nan, y_nan, z_nan, w_nan))
    
    x_vals = x_vals[mask]
    y_vals = y_vals[mask]
    z_vals = z_vals[mask]
    w_vals = w_vals[mask]
    
    x = np.average(x_vals, weights=w_vals)
    y = np.average(y_vals, weights=w_vals)
    z = np.average(z_vals, weights=w_vals)
    
    return (x, y, z)

In [10]:
year_ts = sorted(df_data['YEAR'].unique())
lat_ts = []
long_ts = []

pad = len(str(len(year_ts)))

for i, year in enumerate(year_ts):
    clear_output(wait=True)
    display(f'{i:0{pad}}/{len(year_ts)-1}')
    
    df_year = df_data[df_data['YEAR']==year]
    x_vals, y_vals, z_vals, CO2_vals = [], [], [], []
    for _, row in df_year[['LAT', 'LONG', 'CO2', 'YEAR']].iterrows():
        x, y, z = convert_lat_long_to_cartesian(row['LAT'], row['LONG'])
        x_vals.append(x)
        y_vals.append(y)
        z_vals.append(z)
        CO2_vals.append(row['CO2'])
    
    avg_x, avg_y, avg_z = weighted_average_coordinates(x_vals, y_vals, z_vals, CO2_vals)
    
    avg_lat, avg_long = convert_cartesian_to_lat_long(avg_x, avg_y, avg_z)
    
    lat_ts.append(avg_lat)
    long_ts.append(avg_long)

'270/270'

## Visualise with an animation

Interpolate to make the plot less jittery.

In [11]:
year_interp = np.linspace(min(year_ts), max(year_ts), 1000)
lat_interp = np.interp(year_interp, year_ts, lat_ts)
long_interp = np.interp(year_interp, year_ts, long_ts)

Add extra frames at the end to linger on the final data.

In [12]:
linger = int(len(year_interp)/10)
long_plot = long_interp.copy().tolist()
lat_plot = lat_interp.copy().tolist()
year_plot = year_interp.copy().tolist()
long_plot.extend([long_plot[-1] for _ in range(linger)])
lat_plot.extend([lat_plot[-1] for _ in range(linger)])
year_plot.extend([year_plot[-1] for _ in range(linger)])

Now make the animation.

In [14]:
plt.ioff()
dpi = 85
temp_style = {
    'axes.facecolor': 'black',
    'figure.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'axes.labelsize': 14,
    'ytick.color': 'white',
    'ytick.labelsize': 12
}
with plt.rc_context(temp_style):
    fig = plt.figure(figsize=(1280/dpi, 720/dpi), dpi=dpi)
    ax = fig.add_axes(rect=[0.075, 0.025, 0.8, 0.9], projection=ccrs.PlateCarree())
    xl = (-120, 130)
    yl = (-60, 80)
    ax.set_xlim(xl)
    ax.set_ylim(yl)
    fig.canvas.draw()
    cax = fig.add_axes(
        rect=[0.895, ax.get_position().y0+0.05, 0.025, ax.get_position().height-0.1]
    )
    
    # Map features
    ax.add_feature(OCEAN, facecolor='slategrey')
    ax.add_feature(LAND, facecolor='black', edgecolor='black')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'color': 'white', 'fontsize': 12}
    gl.ylabel_style = {'color': 'white', 'fontsize': 12}

    cmap = plt.cm.get_cmap('YlOrRd')
    norm = plt.Normalize(min(year_plot), max(year_plot))
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, label='Year')
    
    line, = ax.plot([], [], lw=4)
    sc, = ax.plot([], [], marker='o', markersize=10)
    
    y_offset_up = 0.025
    year_text = fig.text(
        ax.get_position().x1,
        ax.get_position().y1+y_offset_up,
        '',
        va='bottom',
        ha='right',
        fontsize=15,
        color='white'
    )
    fig.text(
        ax.get_position().x0,
        ax.get_position().y1+y_offset_up,
        'Evolution of the global centre of human CO$_2$ emissions',
        color='white',
        fontsize=15,
        va='bottom',
        ha='left'
    )
    y_offset_down = 0.01
    fig.text(
        ax.get_position().x0,
        y_offset_down,
        'GitHub: https://github.com/georgeholt1/geospatial_CO2\nTwitter: @GeorgeKHolt',
        va='bottom',
        ha='left',
        color='white',
        fontsize=8
    )
    fig.text(
        ax.get_position().x1,
        y_offset_down,
        'Data sources:    https://ourworldindata.org     https://atcoordinates.info',
        va='bottom',
        ha='right',
        color='white',
        fontsize=8
    )

    def init_anim():
        line.set_data([], [])
        sc.set_data([], [])
        return line,
    
    def animate(i):
        global line, sc
        clear_output(wait=True)
        display(f'{i}/{len(year_plot)-1}')
        x = long_plot[:i+1]
        y = lat_plot[:i+1]
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, cmap=cmap, norm=norm)
        lc.set_linewidth(3)
        lc.set_array(np.array(year_plot))
        line.remove()
        line = ax.add_collection(lc)
        sc.set_data(long_plot[i], lat_plot[i])
        sc.set_color(cmap(norm(year_plot[i])))
        year_text.set_text(f'{int(np.floor(year_plot[i]))}')
        return line,

    anim = FuncAnimation(
        fig, animate, init_func=init_anim, frames=len(year_plot), interval=25
    )
    anim.save('carbon_centre.mp4')
    plt.ion()

'1099/1099'